# One-Class Classification with Tweet Embeddings

In [1]:
# import modules
import numpy as np
import pandas as pd
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.base import TransformerMixin
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
import re
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer

## Load data

In [2]:
xls = pd.ExcelFile('../../../ForLastExpt.xlsx')

In [3]:
class1_train = pd.read_excel(xls, 'Class=1 Training')
class1_tune = pd.read_excel(xls, 'Class=1 Validation')
class1_test_pt1 = pd.read_excel(xls, 'Class=1 Test Part-1')
class1_test_pt2 = pd.read_excel(xls, 'Class=1 Test Part-2')
class0_train = pd.read_excel(xls, 'Class=0 Training')
class0_tune = pd.read_excel(xls, 'Class=0 Validation')
class0_test_pt1 = pd.read_excel(xls, 'Class=0 Test Part-1')
class0_test_pt2 = pd.read_excel(xls, 'Class=0 Test Part-2')

In [4]:
# create outlier target values

# class1_train
outlier = []
for i in class1_train['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_train['outlier_target'] = outlier

# class1_tune
outlier = []
for i in class1_tune['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_tune['outlier_target'] = outlier

# class1_test_pt1
outlier = []
for i in class1_test_pt1['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_test_pt1['outlier_target'] = outlier

# class1_test_pt2
outlier = []
for i in class1_test_pt2['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class1_test_pt2['outlier_target'] = outlier

# class0_train
outlier = []
for i in class0_train['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_train['outlier_target'] = outlier

# class0_tune
outlier = []
for i in class0_tune['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_tune['outlier_target'] = outlier

# class0_test_pt1
outlier = []
for i in class0_test_pt1['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_test_pt1['outlier_target'] = outlier

# class0_test_pt2
outlier = []
for i in class0_test_pt2['Is_Unreliable']:
    if i == 0:
        i = 1
    else:
        i = -1
    outlier.append(i)
class0_test_pt2['outlier_target'] = outlier

In [5]:
train_fold1 = class0_train
tune_fold1 = pd.concat([class1_tune, class0_tune, class1_train], ignore_index = True)
test_fold1 = pd.concat([class1_test_pt1, class1_test_pt2, class0_test_pt1, class0_test_pt2], ignore_index = True)

train_fold2 = class0_test_pt1
tune_fold2 = pd.concat([class1_test_pt2, class1_test_pt1, class0_test_pt2], ignore_index = True)
test_fold2 = pd.concat([class1_tune, class0_tune, class1_train, class0_train], ignore_index = True)

In [6]:
class1_train

,Is_Unreliable,Category,Tweet,outlier_target
0,1,"1, 3, 6, 9",We are living in scary times in Canada. Gov’t ...,-1
1,1,"1, 6, 8, 9","Just as bad in Canada. In fact, our government...",-1
2,1,"1, 4, 9",It was only a matter of time before the mainst...,-1
3,1,"6, 8",Russia's taking no chances: Foreigners infecte...,-1
4,1,"6, 8, 9",Although there is now a presumptive confirmed ...,-1
...,...,...,...,...
95,1,"11, 10",EOIR has ordered immigration court staff to re...,-1
96,1,10,President - who said last night that #coronavi...,-1
97,1,11,"Mashhad, NE #Iran Man collapses possibly due t...",-1
98,1,"10, 11",Pence caught in lies about insurance covering ...,-1


## Derive text vectors from word embeddings

In [7]:
embeddings = pd.read_csv('word_embed_150.csv', index_col = 0)
embeddings

,Comp 1,Comp 2,Comp 3,Comp 4,Comp 5,Comp 6,Comp 7,Comp 8,Comp 9,Comp 10,...,Comp 141,Comp 142,Comp 143,Comp 144,Comp 145,Comp 146,Comp 147,Comp 148,Comp 149,Comp 150
!,0.007566,-0.020793,-0.003942,0.009696,-0.087629,-0.010378,0.166501,-0.003649,0.009360,-0.091673,...,-0.007170,-0.042429,0.008301,-0.004420,-0.005853,-0.025832,-0.004903,0.040462,-0.016921,-0.052454
#,-0.027773,0.009234,0.003370,-0.005749,0.014828,0.003797,-0.005177,0.081886,0.004714,-0.028973,...,0.014547,-0.048027,-0.002176,-0.006363,-0.058348,0.007704,0.021230,-0.005131,0.021984,-0.024996
(,0.002572,-0.010439,-0.005363,-0.001294,-0.033480,-0.007175,-0.019357,-0.003890,0.003273,-0.006273,...,0.008400,-0.007821,-0.017765,-0.026805,-0.001651,0.007308,0.012289,0.003770,-0.015327,-0.003788
),0.001966,-0.009681,-0.005803,-0.001835,-0.035473,-0.007572,-0.017426,-0.003412,0.002609,-0.005599,...,0.008634,-0.004598,-0.016598,-0.027041,-0.001539,0.007361,0.013277,0.002614,-0.015602,-0.004053
",",0.041308,-0.002024,0.064986,-0.034626,-0.013477,-0.082554,-0.000953,0.025784,0.036711,-0.037854,...,-0.041373,-0.019611,-0.037938,0.066940,-0.017248,0.038412,0.016137,0.038610,0.019349,0.005502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
‘,-0.010286,-0.004349,-0.004488,-0.004250,0.052749,-0.022263,0.005821,0.008641,-0.012925,-0.039177,...,-0.030034,0.021322,-0.020181,-0.022382,-0.011236,-0.008548,-0.012283,-0.000855,-0.020855,0.009427
’,-0.016677,0.023127,-0.030161,-0.008522,-0.075953,0.005547,-0.000124,-0.000527,-0.025005,-0.114617,...,-0.002172,-0.005019,-0.071673,0.009328,0.005476,0.017619,-0.007141,-0.014074,-0.048514,-0.064184
“,0.003121,0.013691,0.002170,0.002510,-0.082817,0.014008,0.000128,-0.149647,-0.005229,0.102638,...,0.023585,-0.004590,-0.008809,0.007841,-0.000850,-0.004385,-0.051888,0.037123,-0.007611,0.010452
”,-0.000870,-0.002922,-0.003822,0.002154,-0.063089,0.006624,-0.003883,-0.169095,-0.006157,0.096577,...,0.028698,0.011321,-0.003257,0.001634,-0.008671,-0.000564,-0.071625,0.005545,-0.004277,0.021305


In [8]:
# to convert contractions picked up by word_tokenize() into full words
contractions = {
    "n't": 'not',
    "'ve": 'have',
    "'s": 'is', # note that this will include possessive nouns
    'gonna': 'going to',
    'gotta': 'got to',
    "'d": 'would',
    "'ll": 'will',
    "'re": 'are',
    "'m": 'am',
    'wanna': 'want to'
}

# to convert nltk_pos tags to wordnet-compatible PoS tags
def convert_pos_wordnet(tag):
    tag_abbr = tag[0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
                
    if tag_abbr in tag_dict:
        return tag_dict[tag_abbr]

In [9]:
def get_text_vectors(word_embeddings, # pandas dataframe with row index = vocabulary
                     text_list, # list of strings
                     remove_stopwords = True,
                     lowercase = True,
                     lemmatize = True,
                     add_start_end_tokens = True):
    
    lemmatizer = WordNetLemmatizer()
    
    for k in range(len(text_list)):
        text = text_list[k]
        text = re.sub(r'[_~`@$%^&*[\]+=\|}{\"\'<>/]+', '', text)
        text_vec = np.zeros(word_embeddings.shape[1])
        words = word_tokenize(text)
        tracker = 0 # to track whether we've encountered a word for which we have an embedding (in each tweet)
        
        if remove_stopwords:
            clean_words = []
            for word in words:
                if word.lower() not in set(stopwords.words('english')):
                    clean_words.append(word)
            words = clean_words

        if lowercase:
            clean_words = []
            for word in words:
                clean_words.append(word.lower())

            words = clean_words

        if lemmatize:
            clean_words = []
            for word in words:
                PoS_tag = pos_tag([word])[0][1]

                # to change contractions to full word form
                if word in contractions:
                    word = contractions[word]

                if PoS_tag[0].upper() in 'JNVR':
                    word = lemmatizer.lemmatize(word, convert_pos_wordnet(PoS_tag))
                else:
                    word = lemmatizer.lemmatize(word)

                clean_words.append(word)

            words = clean_words

        if add_start_end_tokens:
            words = ['<START>'] + words + ['<END>']
        
        for i in range(len(words)):
            word = words[i]
            if word in word_embeddings.index.tolist():
                word_embed_vec = word_embeddings.loc[word].values
                if tracker == 0:
                    text_matrix = word_embed_vec
                else:
                    text_matrix = np.vstack((text_matrix, word_embed_vec))
                    
                # only increment if we have come across a word in the embeddings dictionary
                tracker += 1
                    
        for j in range(len(text_vec)):
            text_vec[j] = text_matrix[:,j].mean()
            
        if k == 0:
            full_matrix = text_vec
        else:
            full_matrix = np.vstack((full_matrix, text_vec))
            
    return full_matrix

In [10]:
train_fold1_X = get_text_vectors(embeddings, train_fold1['Tweet'])
train_fold1_y = train_fold1['outlier_target']

tune_fold1_X = get_text_vectors(embeddings, tune_fold1['Tweet'])
tune_fold1_y = tune_fold1['outlier_target']

test_fold1_X = get_text_vectors(embeddings, test_fold1['Tweet'])
test_fold1_y = test_fold1['outlier_target']

train_fold2_X = get_text_vectors(embeddings, train_fold2['Tweet'])
train_fold2_y = train_fold2['outlier_target']

tune_fold2_X = get_text_vectors(embeddings, tune_fold2['Tweet'])
tune_fold2_y = tune_fold2['outlier_target']

test_fold2_X = get_text_vectors(embeddings, test_fold2['Tweet'])
test_fold2_y = test_fold2['outlier_target']

## Binary classification: two-fold CV

In [11]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

In [12]:
# SVC hyperparams to optimize
kernel = ['rbf', 'linear', 'poly', 'sigmoid']

### Fold 1

In [13]:
# training & tuning
models = []
tune_f1 = []
for ker in kernel:
    svc = OneClassSVM(kernel = ker)
    svc.fit(train_fold1_X, train_fold1_y)
    models.append(svc)
    tune_predict = svc.predict(tune_fold1_X)
    f1 = f1_score(tune_fold1_y, tune_predict, average = 'macro')
    tune_f1.append(f1)
        
opt_model = models[tune_f1.index(max(tune_f1))]
opt_model

OneClassSVM(kernel='sigmoid')

In [14]:
tune_f1

[0.6351186990404973, 0.7049689440993788, 0.7, 0.7097267017484299]

In [15]:
# testing
test_scores = dict()
test_predict = opt_model.predict(test_fold1_X)
test_predict_proba = opt_model.decision_function(test_fold1_X)
test_scores['auc'] = [roc_auc_score(test_fold1_y, test_predict_proba, average = 'macro')]
test_scores['accuracy'] = [accuracy_score(test_fold1_y, test_predict)]
test_scores['recall'] = [recall_score(test_fold1_y, test_predict, average = 'macro')]
test_scores['precision'] = [precision_score(test_fold1_y, test_predict, average = 'macro')]
test_scores['f1'] = [f1_score(test_fold1_y, test_predict, average = 'macro')]
test_scores

{'auc': [0.7100510204081633],
 'accuracy': [0.625],
 'recall': [0.625],
 'precision': [0.6621335450995963],
 'f1': [0.6022242967893818]}

### Fold 2

In [16]:
# training & tuning
models = []
tune_f1 = []
for ker in kernel:
    svc = OneClassSVM(kernel = ker)
    svc.fit(train_fold2_X, train_fold2_y)
    models.append(svc)
    tune_predict = svc.predict(tune_fold2_X)
    f1 = f1_score(tune_fold2_y, tune_predict, average = 'macro')
    tune_f1.append(f1)
        
opt_model = models[tune_f1.index(max(tune_f1))]
opt_model

OneClassSVM(kernel='sigmoid')

In [17]:
tune_f1

[0.5581014729950899, 0.564867042707494, 0.5504495504495505, 0.574384236453202]

In [18]:
# testing
test_predict = opt_model.predict(test_fold2_X)
test_predict_proba = opt_model.decision_function(test_fold2_X)
test_scores['auc'].append(roc_auc_score(test_fold2_y, test_predict_proba, average = 'macro'))
test_scores['accuracy'].append(accuracy_score(test_fold2_y, test_predict))
test_scores['recall'].append(recall_score(test_fold2_y, test_predict, average = 'macro'))
test_scores['precision'].append(precision_score(test_fold2_y, test_predict, average = 'macro'))
test_scores['f1'].append(f1_score(test_fold2_y, test_predict))
test_scores

{'auc': [0.7100510204081633, 0.7494897959183674],
 'accuracy': [0.625, 0.6821428571428572],
 'recall': [0.625, 0.6821428571428572],
 'precision': [0.6621335450995963, 0.7248252408841867],
 'f1': [0.6022242967893818, 0.593607305936073]}

### Average results

In [19]:
sum(test_scores['auc'])/2

0.7297704081632653

In [20]:
sum(test_scores['accuracy'])/2

0.6535714285714286

In [21]:
sum(test_scores['recall'])/2

0.6535714285714286

In [22]:
sum(test_scores['precision'])/2

0.6934793929918914

In [23]:
sum(test_scores['f1'])/2

0.5979158013627275